In [1]:
import os
import shutil

from azureml.core.workspace import Workspace
from azureml.core import Experiment
from azureml.core import Environment

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

In [2]:
ws = Workspace.from_config()
ws

Workspace.create(name='ResNet', subscription_id='92c76a2f-0e1c-4216-b65e-abf7a3f34c1e', resource_group='AzureML_UW_ResNet')

In [3]:
# cluster_name = "ML-CPU-test"
cluster_name = "ML-GPU-test"
compute_target = None

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('No compute cluster named {}'.format(cluster_name))
    exit()

Found existing compute target


In [4]:
curated_env_name = 'Resnet50v15-CPU-cluster'
#pytorch_env = Environment.get(workspace=ws, name=curated_env_name)
pytorch_env = Environment.from_conda_specification(name=curated_env_name, file_path='./conda_dependencies.yml')

In [5]:
from azureml.core import ScriptRunConfig
from azureml.core.datastore import Datastore
from azureml.core import Dataset

project_folder = './'
data_path = 'datasets'

datastore = Datastore.get(ws, 'workspaceblobstore')
dataset = Dataset.File.from_files(path=(datastore, data_path))
data_loc = dataset.as_named_input('input').as_mount()
print(data_loc)

In [ ]:
src = ScriptRunConfig(source_directory=project_folder,
                        # command=['ls'],
                        script='train_resnet.py',
                        arguments=[
                          '--num_epochs', 16,
                          '--batch', '32',
                          '--shuffle', 'True',
                          '--dataloc', data_loc,
                          '--output_dir', './outputs',
                        ],
                        compute_target=compute_target,
                        environment=pytorch_env)

run = Experiment(ws, name='Train-Resnet50v15').submit(src)
run.wait_for_completion(show_output=True)

RunId: Train-Resnet50v15_1623928011_bb06fc3c
Web View: https://ml.azure.com/runs/Train-Resnet50v15_1623928011_bb06fc3c?wsid=/subscriptions/92c76a2f-0e1c-4216-b65e-abf7a3f34c1e/resourcegroups/AzureML_UW_ResNet/workspaces/ResNet&tid=72f988bf-86f1-41af-91ab-2d7cd011db47

Streaming azureml-logs/55_azureml-execution-tvmps_7b352698ac18a0a5656560c0fdced9840cff60fdd17d2472546e39fb53e4d3f9_p.txt

2021-06-17T11:07:38Z Running following command: /bin/bash -c sudo blobfuse /mnt/batch/tasks/shared/LS_root/jobs/resnet/azureml/train-resnet50v15_1623928011_bb06fc3c/mounts/workspaceblobstore --tmp-path=/mnt/batch/tasks/shared/LS_root/jobs/resnet/azureml/train-resnet50v15_1623928011_bb06fc3c/caches/workspaceblobstore -o ro --file-cache-timeout-in-seconds=1000000 --cache-size-mb=311641 -o nonempty -o allow_other --config-file=/mnt/batch/tasks/shared/LS_root/jobs/resnet/azureml/train-resnet50v15_1623928011_bb06fc3c/configs/workspaceblobstore.cfg --log-level=LOG_WARNING
2021-06-17T11:07:38Z Successfully mo